<br>

# TJSP

<br>

Michel Metran\
Data: XX.XX.XXXX\
Atualziado em: 31.01.2026


In [ ]:
#!pip3 install tabula

In [ ]:
from datetime import date, datetime
from io import BytesIO
from pathlib import Path

import pandas as pd
import requests
import tabula
from tabula import read_pdf

In [ ]:
from tjsp.paths import data_path

In [ ]:
# Requests
url = 'https://api.tjsp.jus.br/Handlers/Handler/FileFetch.ashx?codigo=177683'
r = requests.get(url, allow_redirects=True)

In [ ]:
# Save PDF file
open(data_path / 'tabela_debitos_judiciais.pdf', 'wb').write(r.content)

<br>

---

## Trata o Dado


In [ ]:
# Read PDF
dfs = tabula.read_pdf(
    input_path=BytesIO(r.content),
    pages='all',
    stream=True,
)
dfs

In [ ]:
def fix_table(df):
    """
    Pega uma tabela que não tem cabeçalho e foi colocado errado a primeira linha como cabeçalho
    e ajusta, criando a primeira linha com os valores do cabeçalho

    :param df: _description_
    :type df: _type_
    :return: _description_
    :rtype: _type_
    """
    # 1. Transformamos o cabeçalho em uma linha e resetamos o índice das colunas
    df_cabecalho = df.columns.to_frame().T
    df_cabecalho.columns = range(len(df.columns))

    # 2. Resetamos os nomes das colunas do DataFrame original para números
    df.columns = range(len(df.columns))

    # 3. Juntamos a "linha do cabeçalho" com os dados e resetamos o índice
    df = pd.concat([df_cabecalho, df], ignore_index=True)
    return df

In [ ]:
list_dfs = []

for n, _ in enumerate(dfs):
    # print(f'{n} dddd')

    # Define a tabela "da vez" e próxima
    df = dfs[n]
    try:
        df_1 = dfs[n + 1]

    except:
        df_1 = pd.DataFrame([{'mes': 0}])
        print('não te proxima!')

    # Rename Coluns
    df = df.rename(
        columns={'Unnamed: 0': 'mes'},
        inplace=False,
        errors='ignore',
    )
    df_1 = df_1.rename(
        columns={'Unnamed: 0': 'mes'},
        inplace=False,
        errors='ignore',
    )

    # Se tem 12 registros, passa
    if len(df) == 12:
        df = df.set_index('mes', inplace=False)
        list_dfs.append(df)

    # Se a tabela da vez e a próxima tem menos que 12 linhas
    elif len(df) < 12 and len(df_1) < 12:
        # display(df.head(2))
        # display(df_1.head(2))

        primeira_coluna_df = list(df.columns)[0]
        primeira_coluna_df_1 = list(df_1.columns)[0]
        if primeira_coluna_df == 'mes' and primeira_coluna_df_1 != 'mes':
            df_1 = fix_table(df=df_1)
            df_1.columns = list(df.columns)[0 : len(df_1.columns)]
            df = pd.concat([df, df_1], ignore_index=True)
            if len(df) == 12:
                # display(df)
                df = df.set_index('mes', inplace=False)
                list_dfs.append(df)

            else:
                raise Exception('Não tem 12 linhas')

    # Aqui é a tabela "adicionada" anteriormente
    elif len(df) < 12 and len(df_1) == 12:
        pass

    else:
        # display(df.head(2))
        # display(df_1.head(2))
        raise Exception('Que condição é essa?')

    print('-' * 100)
    if n == 7:
        # break
        pass

In [ ]:
# df = pd.concat(list_dfs, ignore_index=False)
df = pd.concat(list_dfs, axis=1)
# df = df.set_index('mes', inplace=False)

In [ ]:
# Results
df.to_clipboard(index=False)
df.info()
df.head()

In [ ]:
df = df.stack()
df = pd.DataFrame(df)
df = df.reset_index()

# Results
df.to_clipboard(index=False)
df.info()
df.head()

In [ ]:
# Rename Columns
df = df.rename(
    columns={'level_1': 'ano', 0: 'taxa'},
    inplace=False,
    errors='ignore',
)

# Results
df.to_clipboard(index=False)
df.info()
df.head()

In [ ]:
# Rename Values
dict_mes = {
    'JAN': 1,
    'FEV': 2,
    'MAR': 3,
    'ABR': 4,
    'MAI': 5,
    'JUN': 6,
    'JUL': 7,
    'AGO': 8,
    'SET': 9,
    'OUT': 10,
    'NOV': 11,
    'DEZ': 12,
}

# Ajusta Mês
df = df.replace({'mes': dict_mes})
df['mes'] = df['mes'].astype(int)


# Ajusta Ano
df['ano'] = df['ano'].str.replace(' ', '')
df['ano'] = df['ano'].astype(int)


# Ajusta Datas
df['year'] = df['ano']
df['month'] = df['mes']
df['day'] = 1

df['data'] = pd.to_datetime(df[['year', 'month', 'day']])
df['data_ref'] = df['data'].dt.strftime('%Y-%m')

# Drop
df.drop(['year', 'month', 'day'], axis=1, inplace=True, errors='ignore')

# Sortear
df.sort_values('data', inplace=True)

# Results
df.to_clipboard(index=False)
df.info()
df.head()

In [ ]:
# Ajusta Taxa
df['taxa_string'] = df['taxa']
df['taxa'] = df['taxa'].str.replace('-', '', regex=True)
df['taxa'] = df['taxa'].str.replace(r'\.', '', regex=True)
df['taxa'] = df['taxa'].str.replace(',', '.', regex=True)

df = df[df['taxa'] != '']
df['taxa'] = df['taxa'].astype(float).copy()


# Results
df.to_clipboard(index=False)
df.info()
df.head()

In [ ]:
mask = df['data'] <= pd.Timestamp(
    datetime.today().year, datetime.today().month, 10
)
df = df[mask]

In [ ]:
df = df.reindex(
    columns=[
        'data',
        'data_ref',
        'ano',
        'mes',
        'taxa_string',
        'taxa',
    ],
    # copy=True,
).copy()


df = df.reset_index(
    drop=True,
    inplace=False,
)

In [ ]:

# Results
df.to_clipboard(index=False)
df.info()
df.head()

In [ ]:
# Save "tabela_debitos_judiciais"
df.to_csv(
    data_path / 'tabela_debitos_judiciais.csv',
    index=False,
    decimal=',',
)